In [1]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
from tqdm import tqdm
from collections import Counter
import itertools # 2차원 배열 1차원 배열로
from sklearn.feature_extraction.text import TfidfVectorizer
import os #폴더 이름 가져오기

## 폴더 가져오기

In [2]:
# 고점 or 저점 파일을 파라미터 값으로 넣어 지정돤 폴더에서 csv파일 리스트들을 가져오는 함수
# 파라미터 : 절대경로에 있는 파일
def choice_file(filepath):
    file_list=os.listdir(filepath)
    cnt=0
    for i in file_list:
        i=filepath+"/"+i
        file_list[cnt]=i
        cnt+=1
    return file_list

file_list=choice_file('/Users/home/jp/1일전일봉저점')
file_list

['/Users/home/jp/1일전일봉저점/2022.11.20.csv',
 '/Users/home/jp/1일전일봉저점/2022.06.17.csv',
 '/Users/home/jp/1일전일봉저점/2022.01.21.csv',
 '/Users/home/jp/1일전일봉저점/2022.09.20.csv',
 '/Users/home/jp/1일전일봉저점/2022.10.10.csv',
 '/Users/home/jp/1일전일봉저점/2021.10.26.csv',
 '/Users/home/jp/1일전일봉저점/2022.05.26.csv',
 '/Users/home/jp/1일전일봉저점/2022.12.27.csv',
 '/Users/home/jp/1일전일봉저점/2022.02.20.csv',
 '/Users/home/jp/1일전일봉저점/2022.09.05.csv',
 '/Users/home/jp/1일전일봉저점/.ipynb_checkpoints']

## 필요한 데이터로 변경 (리스트) : 결측치 제거/ column 조정

In [3]:
files=[]
for i in file_list:
    try:
        df=pd.read_csv(i,encoding="UTF-8") # 크롤링한 파일 가져오기
        files.append(df)
    except:
        pass #.ipynb_checkpoints 파일때문

In [4]:
def delete(files):
    result=[]
    for df in files:
        df=df.replace("- dc official App","") # 글 내용없을때 들어가는 "- dc official App" 대체
        df=df.fillna('') # 결측치 제거
        df['data']=df['title']+ " "+df['content'] # 제목+내용을 하나로 합쳐서 'data'로 저장 
        del df['title'],df['content'],df['no'] # 제목,내용,글번호 삭제
        data=df['data']
        result.append(df)
    return result

files=delete(files)

In [5]:
# 기본적인 전처리 과정을 진행하는 함수 
# 파라미터: pandas데이터프레임의 전처리를 원하는 컬럼
def data_processing(files):
    for file in files:
        file['data']=file['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
        file['data']=file['data'].str.replace('^ +', "") # 공백은 empty값으로 변경
        file['data'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
        file['data'].dropna(how='any') # Null 값 제거

data_processing(files)

/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_94038/3334754670.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  file['data']=file['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_94038/3334754670.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  file['data']=file['data'].str.replace('^ +', "") # 공백은 empty값으로 변경


## test- 불용어 파일 리스트로 저장

In [6]:
f=open('stopwords.txt',encoding='utf-8') # 불용어 파일 
words=[]

for i in f:
    words.append(i)
f.close()

words=str(words)[2:-2]

stopwords=words.split(' ')

In [7]:
data=files

## 전처리

In [8]:
def data_processing_stopword(files):
    m=Mecab()
    result=[]
    for file in files:
        rs=[]
        for i in tqdm(file['data'],total=len(file)):
            tokenized_sentence =m.morphs(str(i))
            stopwords_removed_sentence=[word for word in tokenized_sentence if not word in stopwords]
            rs.append(stopwords_removed_sentence)
        result.append(rs)
    return result

In [9]:
test=data_processing_stopword(data)

100%|█████████████████████████████████████| 1929/1929 [00:00<00:00, 3317.12it/s]


## 한글자는 제외

In [10]:
one_word_remove=[]
for result in test:
    temp=[]
    for word in result:
        tmp=[i for i in word if len(i)>1]
        temp.append(tmp)
    one_word_remove.append(temp)

In [11]:
result = []
m = Mecab()

pos_tags = ['NNG', 'NNP', 'VV', 'VA']

for onefile in tqdm(one_word_remove,total=len(one_word_remove)):
    rs = []
    for word in onefile:
        tmp = []
        for i in word:
            if m.pos(i)[0][1] in pos_tags:
                tmp.append(m.pos(i)[0][0])
        rs.append(tmp)
    result.append(rs)


100%|█████████████████████████████████████████████| 9/9 [00:01<00:00,  6.38it/s]


In [12]:
filter_result=[]
for i in result:
    for j in i:
        for k in j:
            if len(k)>1:
                filter_result.append(k)
                

In [13]:
filter_result

['가스',
 '뜬금없',
 '경주마',
 '온돌',
 '가스',
 '폴리',
 '메스',
 '대장',
 '시아',
 '알트',
 '불장',
 '느낌',
 '더니',
 '폴리',
 '가즈',
 '폴리',
 '폴리',
 '발련',
 '아이큐',
 '점마',
 '아이큐',
 '새기',
 '아이큐',
 '설거지',
 '당한',
 '아이큐',
 '모자',
 '폴리',
 '김프',
 '아이큐',
 '담력',
 '테스트',
 '지리',
 '수박',
 '알트',
 '아이큐',
 '아이큐',
 '인성',
 '이큐',
 '아이큐',
 '느낌',
 '양봉',
 '전환',
 '아이큐',
 '개미',
 '털기',
 '아라',
 '아이큐',
 '운지',
 '아이큐',
 '고점',
 '아이큐',
 '자리',
 '아이큐',
 '타이밍',
 '막차',
 '기회',
 '주네',
 '알아보',
 '아이큐',
 '지옥',
 '예상',
 '적중',
 '닭대가리',
 '도지',
 '결강',
 '거래량',
 '니어',
 '계속',
 '아이큐',
 '시아',
 '냄새',
 '아이큐',
 '아침',
 '야수',
 '심장',
 '아이큐',
 '전형',
 '개미',
 '털기',
 '랜드',
 '월드컵',
 '지옥',
 '아이큐',
 '게임',
 '파이',
 '주목',
 '네요',
 '게임',
 '파이',
 '회사',
 '게임',
 '양산',
 '공장',
 '게임',
 '나오',
 '아치',
 '루트',
 '게임',
 '퀄리티',
 '넘사벽',
 '네요',
 '요즘',
 '신규',
 '파트너',
 '똑같',
 '게임',
 '게임',
 '이랑',
 '비교',
 '반응',
 '소식',
 '아쉽',
 '개미',
 '털기',
 '개오',
 '아이큐',
 '양전',
 '아이큐',
 '양봉',
 '전환',
 '좆같',
 '승리',
 '아이큐',
 '개미',
 '가즈',
 '아이큐',
 '장군',
 '양전',
 '부활',
 '아이큐',
 '패닉',
 '흑우',
 '월드컵',
 '역대',
 '사기극',
 '불나방',
 '

## 3차원 배열 2차원으로 변경후 데이터프레임에 저장

In [14]:
sentence=[]
for file in result:
    sen=[]
    for word in file:
        result=' '.join(s for s in word)
        sen.append(result)
    sentence.append(sen)

In [15]:
for i in range(len(files)):
    files[i]['result']=sentence[i]

## 저장

In [17]:
# file_names=os.listdir('/Users/home/jp/1일전일봉저점')
# for df in range(0,len(files)):
#     try:
#         files[df].to_csv('/Users/home/jp/1일전일봉저점/'+"result"+file_names[df],encoding='utf-8')    
#     except:
#         pass

## 빈도수 체크

In [18]:
frequent=Counter(filter_result).most_common()

In [19]:
frequent

[('솔브', 4005),
 ('아이큐', 1729),
 ('루나', 1083),
 ('세력', 1059),
 ('사람', 1014),
 ('애플', 939),
 ('오르', 935),
 ('생각', 927),
 ('파운드', 757),
 ('알트', 746),
 ('상장', 744),
 ('호재', 733),
 ('손절', 711),
 ('세요', 708),
 ('개미', 687),
 ('시작', 676),
 ('이유', 671),
 ('가능', 635),
 ('저점', 607),
 ('정도', 598),
 ('프로', 569),
 ('아서', 562),
 ('페이', 545),
 ('계속', 544),
 ('나오', 537),
 ('이나', 511),
 ('솔브케어', 509),
 ('카바', 500),
 ('아마존', 498),
 ('설거지', 477),
 ('가즈', 472),
 ('리즈', 472),
 ('어제', 469),
 ('스테픈', 463),
 ('인생', 461),
 ('월드컵', 456),
 ('모르', 443),
 ('올리', 442),
 ('무비', 440),
 ('시드', 430),
 ('기사', 429),
 ('청산', 422),
 ('발표', 420),
 ('물량', 418),
 ('거래', 416),
 ('이브', 415),
 ('불장', 409),
 ('얼라', 409),
 ('도지', 383),
 ('펌핑', 382),
 ('떨어지', 380),
 ('내일', 365),
 ('이랑', 357),
 ('절대', 356),
 ('선물', 352),
 ('운지', 350),
 ('부처', 343),
 ('바이', 341),
 ('여자', 340),
 ('웨이브', 333),
 ('단타', 330),
 ('내리', 328),
 ('려고', 327),
 ('나스닥', 326),
 ('폴리곤', 321),
 ('선동', 320),
 ('마지막', 316),
 ('익절', 313),
 ('평단', 311),
 ('발련', 309),
 (

### TF-IDF

In [20]:
corpus=list(files[1]['result'])

In [21]:
s=[]
for i in corpus:
    i=i.replace(" ","")
    s.append(i)

In [22]:
tfidfv=TfidfVectorizer(max_features=10).fit(s)
tfidfv.transform(corpus).toarray()
tfidfv.vocabulary_

{'고생': 0,
 '불장': 2,
 '수고': 5,
 '센티': 3,
 '센티넬': 4,
 '웨이브': 7,
 '운지': 6,
 '청산': 9,
 '무비': 1,
 '유희열표절논란': 8}

In [ ]:
{'떨어지': 2,
 '움직이': 5,
 '기다리': 0,
 '처먹': 6,
 '모으': 4,
 '쳐먹': 7,
 '만드': 3,
 '날리': 1,
 '퍼먹': 9,
 '터지': 8}